In [1]:
# read in the parameters / files / setup

reference_gene = reference.build_gene(options.gene_name)

for sample in range(n_samples):

    sample_gene = copy.deepcopy(reference_gene)

    # decide if resistant or not
    if rand < 0.5:

        # pick from resistant distribution
        n_res_muts = sample_resistant_distribution(n_resistant, variation)

        for res in n_res_muts:

            # pick a resistant mutation and check we haven't already got it

            sample_gene.nucleotide_sequence[sample_gene.nucleotide_number==base_number] = chosen_base
    
    for sus in n_sus_muts:

        # pick a susceptible mutation (without replacement) and apply it

        sample_gene.nucleotide_sequence[sample_gene.nucleotide_number==base_number] = chosen_base

    # add the allele to the list for output
    allele = ''.join(i for i in pnca_reference.amino_acid_sequence)

SyntaxError: incomplete input (2547217384.py, line 5)